In [1]:
from ipyleaflet import Map, basemaps, CircleMarker
from ipywidgets import HTML,Tab, Dropdown, Layout, Image, Box, HBox
import pandas as pd
import sys
sys.path.append("water_security")
from data.labeled.preprocessed import  RISKS_MAPPING as risks
from data.labeled.preprocessed import SEVERITY_MAPPING
import numpy as np
from ipyleaflet import Icon, Marker, MarkerCluster, Popup
import shap
import matplotlib.pyplot as plt
from io import BytesIO
import base64 
m = Map(interpolation='nearest', basemap=basemaps.Stamen.Terrain,world_copy_jump=True)

markers = ()


popup = Popup(
            name='popup',
            close_button=False,
            auto_close=False,
            close_on_escape_key=False,auto_pan=False,
        )
invalid_coords_popup = Popup(
            name='invalid_coords', child=HTML('No city was found near the selection!'),
            close_button=False,
            auto_close=True,
            close_on_escape_key=False,auto_pan=False,
        )

feats_popup = Popup(
            name='feats_popup',
            close_button=True,
            auto_close=False,
            close_on_escape_key=False,auto_pan=True,
            min_width=1000
        )

dpopup = Popup(
            name='download_popup',
            close_button=False,
            auto_close=False,
            close_on_escape_key=False,auto_pan=False
        )
def mouseover_callback(marker,location, city, pred_mask):
    def callback(*args, **kwargs):
        popup.child = populate_per_city(city.city, city.country, city,pred_mask)
        popup.location = location
        m.add_layer(popup)
    return callback

def mouseout_callback():
    def callback(*args, **kwargs):
        try:
            m.remove_layer(popup)
        except:
            pass
    return callback


def plot_shap(values):
    shap.plots.waterfall(values[0], max_display=10, show=False)
    plt.tight_layout()
    shap_plot = plt.gcf()
    
    dpi = 100
    size = [int(x) for x in shap_plot.get_size_inches()*dpi] # size in pixels
    buf = BytesIO()
    shap_plot.savefig(buf,
                format = "png",
                dpi = dpi,bbox_inches = 'tight'
                )
    plt.close()
    img = Image(value=buf.getvalue(),format='png',
    width=f"{size[0]}px",
    height=f"{size[1]}px")
    return img

def show_feats(city, country, risks_dict, pred_mask, shap_values):
    html =  f"""
    <h4><b> {city}, {country} </b></h4>""" #</p>
    
    children = []
    names = []
    for risk in risks:
        if not pred_mask[risk]:
            continue
        if risk not in risks_dict:
            continue
        risk_value = risks_dict[risk]
        if not pd.isnull(risk_value):
            risk_value = np.round(risk_value).astype(int)
            if (risk_value == 0):
                continue
            risk_value = SEVERITY_MAPPING[risk_value]
            
            to_add = HTML(f"""<b>{risks[risk]}:{risk_value}</b>""")
            im = plot_shap(shap_values[risk])
            to_add = im
            children.append(to_add)
            names.append(risks[risk])
    tab = Tab()
    
    for c in children:
        c.layout = Layout(width=f"{1000}px", height=f"{400}px" )
        c.layout.object_fit = 'scale-down'
    tab.children = children
    tab.layout = Layout(width=f"{1050}px", height=f"{400}px" )
    
    for i in range(len(children)):
        tab.set_title(i, names[i])
    return tab  

def populate_per_city(city, country, risks_dict,pred_mask=None, shap_values=None):
    html =  f"""
    <p> <h4><b> {city}, {country} </b></h4>""" #</p>
    for risk in risks:
        if risk not in risks_dict:
            continue
        risk_value = risks_dict[risk]
        if not pd.isnull(risk_value):
            risk_value = np.round(risk_value).astype(int)
            if (risk_value == 0):
                continue
            risk_value = SEVERITY_MAPPING[risk_value]
            to_add = f"""
      <b>{risks[risk]}</b>:{risk_value}
   
    """
            if pred_mask is not None:
                if pred_mask[risk]:
                    to_add = f'<h4 style="color:blue">{to_add}</h4>'
                else:
                    to_add = f' <h4>{to_add}</h4>'
            html += to_add
            
    html += '</p>'
    html = HTML(html)
    
    if shap_values is None:
        return html
    stacked = Tab()
    stacked.children = [html, show_feats(city, country, risks_dict, pred_mask, shap_values)]
    titles = ['Briefly', 'Details']
    for i in range(len(stacked.children)):
        stacked.set_title(i, titles[i])
    return stacked
    


    
from utils.geo import  is_close

In [2]:

from classification.model_handler import ModelHandler, InvalidCoordinates
from data.labeled.preprocessed import RISKS_MAPPING

handler = ModelHandler()

if not handler.is_fitted:
    handler.train()
    
risks_ids = sorted(RISKS_MAPPING)
from data.model.predictions import FILLED_DATASET as dataset ,PREDICTION_MASK as prediction_mask
for (_,city),(_,pred_mask) in zip(dataset.iterrows(),prediction_mask.iterrows()):
    location=(city['latitude'],city['longitude'])
    
    marker = CircleMarker(location=location,fill_color = "blue",color='blue',radius=2)
    marker.on_mouseover(mouseover_callback(marker,location, city, pred_mask))
    marker.on_mouseout(mouseout_callback())
    markers = markers + (marker,)
    
from utils.geo import check_1k_population_density_source
import threading
threads = []
queues = []
def on_download(coords):
    if len(threads) > 1:
        return True
    if len(threads) > 0:
        if threads[0].is_alive():
            m.remove_layer(dpopup)
            dpopup.child = HTML(f"Still Downloading.. {queues[-1].pop()}")
            dpopup.location = coords
            m.add_layer(dpopup)
            return True
        threads.pop()
        queues.pop()
    ret = check_1k_population_density_source()
    if ret is not None:
        dthread, queue = ret
        threads.append(dthread)
        queues.append(queue)
        dpopup.child = HTML("Downloading required resource..")
        dpopup.location = coords
        m.add_layer(dpopup)
        return True
    return False

manually_added_markers = []

def handle_click(**kwargs):
    if kwargs.get('type') == 'click':
        coords = kwargs.get('coordinates')
        check = any(is_close((coords[0],coords[1]), x.location) for x in markers)
        if check:
            return
        if on_download(coords):
            return 
        try:
            m.remove_layer(invalid_coords_popup)
        except:
            pass
        try:
            output,mask, shap_values = handler.test(coords[0], coords[1])

        except InvalidCoordinates:
            invalid_coords_popup.location = coords
            m.add_layer(invalid_coords_popup)
            return
    
        output['city'] = "Close To: " + output['city']
        marker = CircleMarker(location=coords,fill_color = "red",color='red',radius=2)
        check = [is_close(marker.location,mark.location) for mark in manually_added_markers]
        if not any(check):
            
            try:
                m.remove_layer(feats_popup)
            except:
                pass
            marker.on_mouseover(mouseover_callback(marker,coords, output,mask))
            marker.on_mouseout(mouseout_callback())
            manually_added_markers.append(marker)
            m.add_layer(marker)
            feats_popup.child = populate_per_city(output.city, output.country, output, mask, shap_values)
            feats_popup.location = marker.location
            m.add_layer(feats_popup)
        else:
            to_rem = [cnt for cnt,(m,c) in enumerate(zip(manually_added_markers,check)) if c][0]
            m.remove_layer(manually_added_markers[to_rem])
            manually_added_markers.pop(to_rem)

m.on_interaction(handle_click)
m.add_layer(MarkerCluster(markers = markers))

Loaded model from water_security/data/model/model.pkl.


## Instructions
### Click on clouds of cities to expand or hover over single points to view risks
- Risks shown in black are result of ground truth
- Risks shown in blue are result of prediction
- The documented risks are the following:
 - Higher water prices
 - Inadequate or aging infrastructure
 - Increased water stress or scarcity
 - Declining water quality
 - Increased water demand
 - Regulatory
 - Energy supply issues
- The Risks can have one of the following levels: Less Serious, Serious, Extremely Serious. If there is a risk not mentioned, then it has been determined that there is no such risk for the prediction point.

### Click on the map to get online prediction. Click the created red dot to remove.
- In this view the features SHAP values can also be seen for the predictions by clicking on the "Details" tab.
- The details can only be accessed only on the first click. Once you click on a different location or close the window, only hover can be done for that point.
- To close the window there is an `X` on the top right, you might need to drag the map to reach it.
- There will be a short delay between the click and the appearance of the red dot/information

**(For online prediction, a file containing population densities is required to be downloaded, which is ~900MB in size)**


In [3]:
m.layout.width = '80%'
m.layout.height = '1000px'
m.zoom=2.2
m.center = (0, 0)
display(m)

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

ntree_limit is deprecated, use `iteration_range` or model slicing instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.
Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations. 
